In [ ]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
from pycmqlib3.utility import dbaccess, dataseries, misc
from pycmqlib3.analytics.tstool import *
from pycmqlib3.analytics.btmetrics import *
from pycmqlib3.analytics.backtest_utils import *
from pycmqlib3.strategy.signal_repo import *

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
matplotlib.rcParams['figure.figsize'] = (12, 8)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

# load historical data

In [ ]:
from misc_scripts.update_fut_prices import load_saved_fut
tday = datetime.date(2025, 3, 14)

df = load_saved_fut(tday, freq='d')
#df = load_cnc_fut(tday, type='cnc')

start_date = df.index[0]
end_date = tday

cdates = pd.date_range(start=start_date, end=tday, freq='D')
bdates = pd.bdate_range(start=start_date, end=end_date, freq='C', holidays=misc.CHN_Holidays)

In [ ]:
asset_pairs = [
    ('T', 'TF'), ('TF', 'T'), ('TS', 'TF'), ('T', 'TL'),
]

roll_win = 244
beta_ret_dict = {}
betas_dict = {}
for trade_asset, index_asset in asset_pairs:
    asset_df = df[[(index_asset+'c1', 'close'), (trade_asset+'c1', 'close')]].copy(deep=True).droplevel([1], axis=1)
    asset_df = asset_df.dropna(subset=[trade_asset+'c1']).ffill()

    for asset in asset_df:
        asset_df[f'{asset}_pct'] = asset_df[asset].pct_change().rolling(1).mean()
    asset_df['beta'] = asset_df[f'{index_asset}c1_pct'].rolling(244).cov(asset_df[f'{trade_asset}c1_pct'])/asset_df[f'{index_asset}c1_pct'].rolling(roll_win).var()
    key = '_'.join([trade_asset, index_asset])
    asset_df[key] = asset_df[trade_asset+'c1'].pct_change() - asset_df['beta'] * asset_df[index_asset+'c1'].pct_change().fillna(0)
    beta_ret_dict[key] = asset_df[key].dropna()
    betas_dict[key] = asset_df['beta']


In [ ]:
spot_df = load_fun_data(tday)


In [ ]:
spot_dict = {}

vol_win = 20
product_list = list(set([col[:-2] for col in df.columns.get_level_values(0).unique()]))

for asset in product_list:    
    spot_dict[f'{asset}_px'] = df[(asset+'c1', 'close')]
    spot_dict[f'{asset}_logret'] = np.log(df[(asset+'c1', 'close')]).dropna().diff()
    spot_dict[f'{asset}_pctchg'] = np.log(df[(asset+'c1', 'close')]).dropna().pct_change()        
    if (asset+'c2', 'close') not in df.columns:
        print(asset)
        continue
    spot_dict[f'{asset}_ryield'] = (np.log(df[(asset+'c1', 'close')]) - np.log(df[(asset+'c2', 'close')]) - 
                                  df[(asset+'c1', 'shift')] + df[(asset+'c2', 'shift')])/(df[(asset+'c2', 'expiry')] - df[(asset+'c1', 'expiry')]).dt.days*365.0 + spot_df['shibor_1m'].dropna().ewm(1).mean()/100
    spot_dict[f'{asset}_logret2'] = np.log(df[(asset+'c2', 'close')]).dropna().diff()
    spot_dict[f'{asset}_basmom'] =spot_dict[f'{asset}_logret'] - spot_dict[f'{asset}_logret2']        
    spot_dict[f'{asset}_pctvol'] = spot_dict[f'{asset}_pctchg'].dropna().rolling(vol_win).std()
    spot_dict[f'{asset}_basmom5'] = spot_dict[f'{asset}_basmom'].dropna().rolling(5).sum()
    spot_dict[f'{asset}_basmom10'] = spot_dict[f'{asset}_basmom'].dropna().rolling(10).sum()
    spot_dict[f'{asset}_basmom20'] = spot_dict[f'{asset}_basmom'].dropna().rolling(20).sum()
    spot_dict[f'{asset}_basmom60'] = spot_dict[f'{asset}_basmom'].dropna().rolling(60).sum()

keys = [key for key in spot_df.columns if key not in spot_dict]
spot_df = pd.concat([spot_df[keys], pd.DataFrame(spot_dict)], axis=1)

In [ ]:
from pandas.tseries.offsets import CustomBusinessDay
def create_holiday_window_series(index, holidays, pre_days, post_days):
    chn_bday = CustomBusinessDay(holidays=pd.to_datetime(CHN_Holidays))
    series = pd.Series(0, index=index)        
    for holiday in holidays:        
        start = holiday + pre_days * chn_bday
        end = holiday + post_days * chn_bday               
        series.loc[(series.index >= start) & (series.index <= end)] = 1
    return series

start_date = datetime.datetime(2014, 5, 1)

# Get the current year and month
current_date = datetime.datetime.now()
latest_contract = current_date.strftime("%Y%m")  # Latest contract in YYYYMM format

# Generate contract months from May 2020 to the latest contract
contract_months = []
while start_date.strftime("%Y%m") <= latest_contract:
    contract_months.append(int(start_date.strftime("%Y%m")))
    # Move to the next month
    start_date += datetime.timedelta(days=32)
    start_date = start_date.replace(day=1)  # Ensure we stay at the first day of the month

opt_expiries = [misc.get_opt_expiry("i"+str(cont)[-4:], cont) for cont in contract_months]


In [ ]:
for asset in empiric_assets:
    spot_df[f"{asset}_reaction"] = np.sign(df[(asset+'c1', 'close')] - df[(asset+'c1', 'settle')])*(df[(asset+'c1', 'close')].shift(-1)-df[(asset+'c1', 'close')]) #/df[(asset+'c1', 'close')].diff().std()

# feature study

In [ ]:
vol_win=20
pnl_tenors = ['6m', '1y', '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y', '10y']
insample = "2026-01-01"

empiric_assets = [
    'T', 'TF', 'TS', 'TL',
    #'rb', 'hc', 'j', 'jm', 'i', 'v', 'FG', 'SM', 'SF', 'SA', 
    #'cu', 'al', 'zn', 'ni', 'sn', 'ss', 'pp', 'sc', 'lu'
]

traded_price = 'a1535'
df_pxchg = pd.DataFrame(index=df.index)

for asset in empiric_assets:
    if '_' in asset:
        df_pxchg[asset] = beta_ret_dict[asset].dropna()[:insample]
    elif '-' in asset:
        for sub_asset in asset.split('-'):
            if sub_asset not in empiric_assets:
                df_pxchg[sub_asset] = df[(sub_asset+'c1', traded_price)].dropna().pct_change()[:insample]                 
    else:
        if asset in ["T", "TF", "TS", "TL", "IF", "IH", "IC", "IM"]:
            if traded_price in ['n305', 'n310', 'n315', 'n450', 'a1505', 'a1510', 'a1515']:
                ts_px = df[(asset+'c1', 'a1535')]
            else:
                ts_px = df[(asset+'c1', traded_price)]
            flag = ts_px.isna()
            ts_px.loc[flag] = df[(asset+'c1', 'd_twap')].loc[flag]
        else:
            ts_px = df[(asset+'c1', traded_price)]
            if traded_price in ['n305', 'n310', 'n315', 'n450']:
                flag = ts_px.isna()
                ts_px.loc[flag] = df[(asset+'c1', 'a1505')].loc[flag]
            elif traded_price in ['n_twap']:
                flag = ts_px.isna()
                ts_px.loc[flag] = df[(asset+'c1', 'd_twap')].loc[flag]        
                
        df_pxchg[asset] = ts_px.dropna().pct_change()[:insample]        
vol_df = df_pxchg.rolling(vol_win).std()

In [ ]:
vol_df2 = pd.DataFrame(index=df.index)
for asset in df_pxchg.columns:
    vol_df2[asset] = robust_vol_calc(df_pxchg[asset])

In [ ]:
spot_df['pmi_steel_order_inv_ratio'] = spot_df['pmi_cn_steel_new_order']/spot_df['pmi_cn_steel_inv']
spot_df['pmi_steel_order_prod_ratio'] = spot_df['pmi_cn_steel_new_order']/spot_df['pmi_cn_steel_prod']
spot_df['pmi_steel_order_rminv_ratio'] = spot_df['pmi_cn_steel_new_order']/spot_df['pmi_cn_steel_rm_inv']

spot_df['pmi_order_rminv_ratio'] = spot_df['pmi_cn_manu_new_order']/spot_df['pmi_cn_manu_rm_inv']
spot_df['pmi_order_purchase_ratio'] = spot_df['pmi_cn_manu_new_order']/spot_df['pmi_cn_manu_purchase']

spot_df['pmi_lgsc_order_inv_ratio'] = spot_df['pmi_lgsc_steel_tot_order']/spot_df['pmi_lgsc_steel_inv']
spot_df['pmi_lgsc_purchase_inv_ratio'] = spot_df['pmi_lgsc_steel_purchase_exp']/spot_df['pmi_lgsc_steel_inv']

In [ ]:
cutoff='2010-01-01'

signal_cap = [-2, 2]
chg_func = 'diff'
bullish = False
vol_win = 24

by_asset = False

signal_func = 'zscore'
param_rng = [48, 60, 1]
#feature = 'exch_warrant'
#feature = "sm_margin_north"
#feature = "sf_neimeng_margin"
#feature = "cnh_cny_spd2"
#feature = 'base_sw_csi500_ret'
#feature = 'glass_sw_csi500_ret'
#feature = 'sf_neimeng_cost'
#feature = 'mn_44_gabon_tj'
#feature = "phycarry"
# feature = 'rebar_sales_inv_ratio'
#feature = "pmi_cn_cons_bus_exp"
#'ni_npi_10-15_sh' #'ni_nis_cjb_spot' # 'ni_mhp_34_ports' #"ni_1.5conc_spot_rz" #'ni_npi_10-15_sh'
#feature = 'ppi_cpi_mom_spd' #'pmi_cn_steel_all' #
feature="pmi_cn_manu_all"
#feature='fxbasket_cumret'

freq=''

signal_df = pd.DataFrame(index=df_pxchg.index)

for asset in empiric_assets:
    #feature_ts = df_pxchg[asset].cumsum()
    #feature_ts = df[(asset+'c1', 'close')].dropna() #.pct_change() 
    if by_asset:
        asset_feature = f"{asset}_{feature}"
    else:
        asset_feature = feature
    feature_ts = spot_df[asset_feature].dropna() #.rolling(12).sum()
    #feature_ts = feature_ts.diff(12)
    #feature_ts = feature_ts.ewm(1).mean()
    #feature_ts.index = feature_ts.index + pd.DateOffset(days=9) + chn_bday * 1
    #feature_ts = np.log(feature_ts)
    #feature_ts = feature_ts.rolling(100).sum()
    #feature_ts = (1+stock_pct_chg[["SPY.P"]].mean(axis=1)).cumprod()
    #feature_ts = (1+stock_pct_chg[['SPY.P']].mean(axis=1)).cumprod()
    #feature_ts = spot_df[asset_feature].ffill().reindex(index=df_pxchg.index)
    #feature_ts = beta_ret_dict[asset].dropna().cumsum()
    #feature_ts = spot_df[feature].ffill().reindex(index=pd.date_range(start=df.index[0], end=df.index[-1], freq=freq)).ffill().dropna()    
    #feature_ts = df[(asset, 'c1', 'close')].dropna()
    #ticker, param_rng = feature_map[asset]    
    #feature_ts = (spot_df[beta_feature_map[asset][0]]/spot_df[beta_feature_map[asset][1]]).dropna()
    #signal_ts = (feature_ts.ewm(8).mean() - feature_ts.ewm(30).mean())/feature_ts.diff().ewm(30).std()
#     feature_ts = beta_residual(
#         spot_df[feature].dropna(),
#         df_pxchg[asset].dropna().cumsum(),
#         beta_win=120, 
#         chg_func='diff') 
#     feature_ts = beta_residual(
#         spot_df[beta_feature_map[asset][0]].dropna(),
#         spot_df[beta_feature_map[asset][1]].dropna(),
#         beta_win=250, 
#         chg_func='pct_change') 
    #feature_ts = np.log(feature_ts)
#     if asset in ['cu', 'al']:
#         #param_rng = [1, 2, 1]
#         feature_ts = feature_ts.rolling(20).mean()
#     else:
#         #param_rng = [1, 2, 1]
#         feature_ts = feature_ts.rolling(2).mean()
    #feature_ts = feature_ts.reindex(index=cdates).ffill().reindex(index=bdates)
    #feature_ts = feature_ts.ewm(10).mean()
    #feature_ts = yoy_generic(feature_ts, label_func=lunar_label, group_col='label_day', func=chg_func)[asset_feature]    
    signal_ts = calc_conv_signal(feature_ts, signal_func=signal_func, param_rng=param_rng, signal_cap=signal_cap, vol_win=vol_win)
    #signal_ts = feature_ts
    #signal_ts = 4*signal_ts - 3*signal_ts.shift(1)
    #signal_ts = np.sign(signal_ts)  
    #signal_ts = conv_ewm(feature_ts, [2, 6, 2], [8, 32, 4], vol_win=60)    
    #signal_ts = zscore_roll(feature_ts, 120)
    #signal_ts = ewmac(feature_ts, 8, 16, vol_win=0)
    #signal_ts = pd.Series(1, index=df_pxchg.index)
    #signal_ts.loc[signal_ts.index.month.isin([1, 2, 3, 4, 5, 6, 7, 12])] = 1
    #signal_ts.loc[signal_ts.index.day.isin(range(16, 32))] += 1
    #signal_ts = zscore_roll(feature_ts, 60)
    #signal_ts = signal_hysteresis(signal_ts, 1.2, 0.6, False)
    #signal_ts = seasonal_score(feature_ts.to_frame(), backward=10, forward=10, rolling_years=5, min_obs=10)
    # signal_ts = seasonal_score(feature_ts.to_frame(), backward=15, forward=15, rolling_years=3, min_obs=30)
    #signal_ts = create_holiday_window_series(df.index, opt_expiries, 1, 3) # - create_holiday_window_series(df.index, opt_expiries, 3, 4)
    if not bullish:
        signal_ts = -signal_ts    
    
    signal_ts = signal_ts.reindex(index=cdates).ffill().reindex(index=df_pxchg.index)
    #signal_ts = signal_ts.ewm(3).mean()    
    #signal_ts.loc[signal_ts.index.month.isin([1, 2])] = 0
    #signal_ts = signal_hump(signal_ts, 0.2)
    if '-' in asset:
        sub_assets = asset.split('-')
        if sub_assets[0] in signal_df.columns:
            signal_df[sub_assets[0]] += signal_ts
        else:
            signal_df[sub_assets[0]] = signal_ts
        if sub_asset[1] in signal_df.columns:
            signal_df[sub_assets[1]] -= signal_ts
        else:
            signal_df[sub_assets[1]] = -signal_ts            
    else:
        signal_df[asset] = signal_ts

#signal_df = xs_demean(signal_df)
#signal_df = signal_df + xs_demean(signal_df)
#signal_df = signal_buffer(signal_df, 0.1)

holding = generate_holding_from_signal(signal_df, vol_df, risk_scaling=1.0, asset_scaling=False)

bt_metrics = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                         returns=df_pxchg[empiric_assets][cutoff:], 
                         shift_holdings=1)
trading_cost = dict([(asset, 0.6e-4) if asset in ["T", "TF", "TS", "TL"] else (asset, 2e-4) for asset in holding.columns])

bt_metrics_w_cost = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                                returns=df_pxchg[empiric_assets][cutoff:], 
                                shift_holdings=1,
                                cost_dict=trading_cost)

pnl_stats = bt_metrics.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors)
pnl_stats_w_cost = bt_metrics_w_cost.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors)

print("SR after cost:\n", pnl_stats_w_cost['sharpe'])
print(pnl_stats_w_cost['asset_sharpe_stats'])
print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats_w_cost['turnover'], pnl_stats_w_cost['pnl_per_trade']))

print("SR before cost:\n", pnl_stats['sharpe'])
print(pnl_stats['asset_sharpe_stats'])
print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats['turnover'], pnl_stats['pnl_per_trade']))

iplot(pnl_stats_w_cost['portfolio_cumpnl'], title='portfolio pnl with cost')
iplot(pnl_stats_w_cost['asset_cumpnl'], title='asset pnl with cost')

iplot(pnl_stats['portfolio_cumpnl'], title='portfolio pnl wo cost')
iplot(pnl_stats['asset_cumpnl'], title='asset pnl wo cost')


In [ ]:
pnl = pnl_stats["portfolio_pnl"]['total']
pnl = pnl[pnl.abs()>0].to_frame()
print("zscore=%s" % (pnl.mean()/pnl.std()*np.sqrt(len(pnl)/len(df_pxchg.loc[pnl.index[0]:pnl.index[-1]]))*16))


In [ ]:
calc_perf_by_tenors(pnl['total'],tenors=['1y', '2y', '3y', '5y', '7y', '10y'], metric='sharpe')

In [ ]:
lead_lag_config = {
    'll_left': -20,
    'll_right': 120,
    'll_spacing': 5,
    'll_sub_win': [(datetime.date(2008, 1, 1), datetime.date(2018, 12, 31)), 
                   (datetime.date(2019, 1, 1), datetime.date(2024, 12, 31)),],
}

ll_keys = ['fullsample'] + ['%s:%s' % (sd.strftime('%Y-%b-%d'), ed.strftime('%Y-%b-%d')) for sd, ed in lead_lag_config['ll_sub_win']]

#tilt_timing = bt_metrics.tilt_timing(tilt_rolling_window=1*244) # default 3 years  tilt_rolling_window = 3 * 244 

seasonal_pnl = bt_metrics.seasonal_pnl()
cumpnl = seasonal_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
print('seasonal sharpe stats\n', seasonal_pnl['sharpe_stats'])
plt.grid()
plt.title('monthly pnl')
plt.show()


monthday_pnl = bt_metrics.monthday_pnl()
cumpnl = monthday_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('monthday sharpe stats\n', monthday_pnl['sharpe_stats'])
plt.grid()
plt.title('monthday pnl')
plt.show()


week_pnl = bt_metrics.week_pnl()
cumpnl = week_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('week sharpe stats\n', week_pnl['sharpe_stats'])
plt.grid()
plt.title('weekday pnl')
plt.show()


annual_pnl = bt_metrics.annual_pnl()
cumpnl = annual_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('annual sharpe stats\n', annual_pnl['sharpe_stats'])
plt.grid()
plt.title('annual pnl')
plt.show()

annual_pnl['cumlog_pnl'].mean(axis=1).plot()
plt.grid()
plt.title('annual averaged profile')
plt.show()

# turnover = bt_metrics.turnover()
# print(turnover)

In [ ]:
vol_win=20
pnl_tenors = ['6m', '1y', '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y', '10y']
insample = "2026-01-01"

empiric_assets = [
#     'cu', 'zn', 'al', 'ni', 'ss',
#     'rb', 'hc', 'SM', 'SF', 'FG', 'v',
#     'y', 'OI', 'm', 'RM',
#     'l', 'MA', 'pp', 'TA', 'eg',
#     'au', 'ag', 
    'T', 'TF', 'TL',
    #'TL', 'TS',
#     'IF', 'IC', 'IH'
]

traded_price = 'a1535'
df_pxchg = pd.DataFrame(index=df.index)

for asset in empiric_assets:
    if '_' in asset:
        df_pxchg[asset] = beta_ret_dict[asset].dropna()[:insample]
    elif '-' in asset:
        for sub_asset in asset.split('-'):
            if sub_asset not in empiric_assets:
                df_pxchg[sub_asset] = df[(sub_asset+'c1', traded_price)].dropna().pct_change()[:insample]                 
    else:
        ts_px = df[(asset+'c1', traded_price)]
        if traded_price in ['n305', 'n310', 'n315', 'n450']:
            flag = ts_px.isna()
            ts_px.loc[flag] = df[(asset+'c1', 'a1505')].loc[flag]
        df_pxchg[asset] = ts_px.dropna().pct_change()[:insample]        
vol_df = df_pxchg.rolling(vol_win).std()


In [ ]:
feature_setup = {
    'bond_mr_st_qtl': [['T', 'TF'],
                       ['px', 'qtl', [2, 5, 1], 'df1', 'pct_change', False, '', "ema10|buf0.1", 120, [-2,2]]],
    'bond_tf_lt_qtl': [['T', 'TF', 'TL'],
                       ['px', 'qtl', [230, 250, 2], '', '', True, '', "buf0.1", 120, [-2,2]]],
    'bond_tf_st_eds': [['T', 'TF', 'TL', "TS"],
                       ['px', 'ema_dff_sgn', [20, 40, 2], '', '', True, '', "ema5", 120, [-2,2]]],
    'bond_carry_ma': [['T', 'TL'],
                      ['ryield', 'ma', [1, 2, 1], '', '', True, '', "", 120, [-2,2]]],
    "bond_fxbasket_zs": [['T', 'TF', 'TL', "TS"],
                    ['fxbasket_cumret', 'zscore', [480, 520, 2], '', '', True, '', '', 120, [-2,2]]],        
}
    

standard signal

In [ ]:
cutoff='2016-01-01'

feature_names = [
    ['bond_mr_st_qtl', 2.406], # 
    ['bond_tf_lt_qtl', 0.1], # 0.38
    ['bond_carry_ma', 0.05], # 0.22
    ['bond_tf_st_eds', 0.138],
    ["bond_fxbasket_zs", 0.097],
]

signal_dict = {}
signal_df = pd.DataFrame(0, columns=empiric_assets, index=cdates)

for feature_name, weight in feature_names:
    signal_assets = feature_setup[feature_name][0]
    feature, signal_func, param_rng, proc_func, chg_func, bullish, freq, post_func, vol_win, signal_cap = feature_setup[feature_name][1] 
    if feature in ['sinv', 'base_phybas', 'prem_bonded_warrant', 'prem_bonded_cif', 
                   'tc', 'phycarry', 'ryield', 'basmom5', 'basmom10', 'basmom20', 'basmom40', 'basmom60', 'basmom120', 
                   'base_inv', 'inv_shfe_d', 'inv_lme_total', 'inv_exch_d', 'px']:
        sig_df = pd.DataFrame(0, columns=empiric_assets, index=cdates)
        for asset in empiric_assets:
            if feature == 'base_inv':
                asset_feature = base_inv.get(asset, f"{asset}_{feature}")
            else:
                asset_feature = f"{asset}_{feature}"
            if (asset not in signal_assets) or (asset_feature not in spot_df.columns):
                sig_df[asset] = np.nan
                continue
            if feature in ['base_phybas']:
                if asset in ['cu', 'al']: 
                    proc_func = 'sma20'
                else:
                    proc_func = 'sma2'
            signal_ts = calc_funda_signal(spot_df, asset_feature, signal_func, param_rng,
                                          proc_func=proc_func, chg_func=chg_func, bullish=bullish,
                                          freq=freq, signal_cap=signal_cap, bdates=bdates,
                                          post_func=post_func, vol_win=vol_win)
            sig_df[asset] = signal_ts
        sig_df = sig_df.reindex(index=cdates).ffill().reindex(index=df_pxchg.index)
        if "xdemean" in feature_name:
            sig_df = xs_demean(sig_df)
        elif "xscore" in feature_name:
            sig_df = xs_score(sig_df)
        elif "xrank" in feature_name:
            sig_df = xs_rank(sig_df, 0.2)

    else:
        sig_ts = calc_funda_signal(spot_df, feature, signal_func, param_rng,
                                   proc_func=proc_func, chg_func=chg_func, bullish=bullish,
                                   freq=freq, signal_cap=signal_cap, bdates=bdates,
                                   post_func=post_func, vol_win=vol_win)
        sig_df = pd.DataFrame(columns=empiric_assets, index=cdates)
        for asset in empiric_assets:
            if asset in signal_assets:
                sig_df[asset] = sig_ts
            else:
                sig_df[asset] = 0
        sig_df = sig_df.reindex(index=cdates).ffill().reindex(index=df_pxchg.index)
    last_func = post_func.split('|')[-1]
    if 'buf' in last_func:
        buffer_size = float(last_func[3:])
        sig_df = signal_buffer(sig_df, buffer_size)
    elif 'bfc' in last_func:
        buffer_size = float(last_func[3:])
        sig_df = signal_cost_optim(sig_df, 
                                   buffer_size, 
                                   vol_df, 
                                   cost_dict = dict([(asset, 1e-4) for asset in empiric_assets]))
    sig_df = sig_df.reindex(index=cdates).ffill().reindex(index=df_pxchg.index).fillna(0)
    signal_dict[feature_name] = sig_df * weight
    signal_df = signal_df + signal_dict[feature_name]
    
signal_dict['combo'] = signal_df.dropna().ffill()

In [ ]:
pnl_df = pd.DataFrame(index=df_pxchg.index)
for signal_name in signal_dict:
    signal_df = signal_dict[signal_name]
    holding = generate_holding_from_signal(signal_df, vol_df, risk_scaling=1, asset_scaling=False)    
    
    bt_metrics = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                             returns=df_pxchg[empiric_assets][cutoff:], 
                             shift_holdings=1)

    bt_metrics_w_cost = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                                    returns=df_pxchg[empiric_assets][cutoff:], 
                                    shift_holdings=1,
                                    cost_dict=simple_cost(holding.columns, trd_cost=1e-4))

    pnl_stats = bt_metrics.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors)
    pnl_stats_w_cost = bt_metrics_w_cost.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors)
    pnl_df[signal_name] = pnl_stats_w_cost['portfolio_pnl']['total']
    
    print("signal=%s\n" % signal_name)
    print("SR after cost:\n", pnl_stats_w_cost['sharpe'])
    print(pnl_stats_w_cost['asset_sharpe_stats'])
    print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats_w_cost['turnover'], pnl_stats_w_cost['pnl_per_trade']))

    print("SR before cost:\n", pnl_stats['sharpe'])
    print(pnl_stats['asset_sharpe_stats'])
    print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats['turnover'], pnl_stats['pnl_per_trade']))

    iplot(pnl_stats_w_cost['portfolio_cumpnl'], title='portfolio pnl with cost')
    iplot(pnl_stats_w_cost['asset_cumpnl'], title='asset pnl with cost')

    iplot(pnl_stats['portfolio_cumpnl'], title='portfolio pnl wo cost')
    iplot(pnl_stats['asset_cumpnl'], title='asset pnl wo cost')

corr_cutoff = '2017-01-01'
print("std:\n%s\n" % pnl_df[corr_cutoff:].std(axis=0))

pnl_w_df = pnl_df.drop(columns=['combo']).resample('W').sum()
print("corr:\n%s\n" % pnl_w_df[corr_cutoff:].corr())

In [ ]:
import pypfopt
from pypfopt import plotting
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import expected_returns

dpnl = pnl_df['2017-01-01':].drop(columns=['combo'])
strat_std = dpnl.std()
dpnl = dpnl/dpnl.std()

max_sharpe = False

if max_sharpe:
    mu = expected_returns.mean_historical_return(dpnl, returns_data=True, frequency=244, compounding=False)
else:
    mu = None

S = risk_models.CovarianceShrinkage(dpnl, returns_data=True, frequency=244).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds= (0, 1)) 

fact_idx = {}
for fact in dpnl.columns:
    fact_idx[fact] = ef.tickers.index(fact)

#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] >= 0.3)
#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] <= 0.4)
#ef.add_constraint(lambda w: w[fact_idx['ryield_ema_ts']] + w[fact_idx['ryield_ema_xdemean']] >= 0.3)
# ef.add_constraint(lambda w: w[fact_idx['tsmom']] + w[fact_idx['macro2']] <= 0.24)

#ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
if max_sharpe:
    port_weights = ef.max_sharpe()
else:
    port_weights = ef.min_volatility()

#port_weights = ef.clean_weights()
port_weights = pd.Series(port_weights)
print("port_weights=\n%s\n" % port_weights)
final_weights = port_weights.div(strat_std)
print("final_weights=\n%s\n" % final_weights)
# mu = expected_returns.mean_historical_return(df)
# S = risk_models.sample_cov(df)

# # Optimize for maximal Sharpe ratio
# ef = EfficientFrontier(mu, S)
# weights = ef.max_sharpe()
# ef.portfolio_performance(verbose=True)

In [ ]:
ls_pnl[key]['portfolio_cumpnl']

In [ ]:
lead_lag_config = {
    'll_left': -20,
    'll_right': 120,
    'll_spacing': 5,
    'll_sub_win': [(datetime.date(2008, 1, 1), datetime.date(2018, 12, 31)), 
                   (datetime.date(2019, 1, 1), datetime.date(2024, 12, 31)),],
}

ll_keys = ['fullsample'] + ['%s:%s' % (sd.strftime('%Y-%b-%d'), ed.strftime('%Y-%b-%d')) for sd, ed in lead_lag_config['ll_sub_win']]


ll_left = lead_lag_config['ll_left']
ll_right = lead_lag_config['ll_right']
spacing = lead_lag_config['ll_spacing']

leadlag_df = bt_metrics.lead_lag(ll_limit_left=ll_left, 
                                 ll_limit_right=ll_right,
                                 ll_sub_windows=lead_lag_config['ll_sub_win'])

fig, ax = plt.subplots(len(ll_keys), 1)
fig.set_figheight(15)
fig.set_figwidth(10)

for i, key in enumerate(ll_keys):
    ts = leadlag_df['leadlag_sharpes'].loc[key]
    ts.plot(kind='bar', ax = ax[i], title = f'lead_lag: {key}')
    new_ticks = np.linspace(ll_left, ll_right, (ll_right-ll_left)//spacing+1)
    ax[i].set_xticks(np.interp(new_ticks, ts.index, np.arange(ts.size)))
    ax[i].set_xticklabels(new_ticks)
    ax[i].axvline(x=-ll_left, color='red', linestyle='--')
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111)
ls_pnl = bt_metrics.long_short_pnl()
for key in ls_pnl:
    ax.plot(ls_pnl[key]['portfolio_cumpnl'], '-', label=key)
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, bbox_to_anchor=(1.04, 1), loc='upper left')
ax.grid()
plt.title("long-short pnl")
plt.show()

lagged = bt_metrics.lagged_pnl(lags=[1, 5, 10, 20, 30, 60, 75, 80])
lagged['cumpnl'].plot()
#print('lagged PNL\n', lagged['sharpe'])
plt.grid()
plt.title('lagged pnl')
plt.show()

smoothed = bt_metrics.smoothed_pnl(smooth_hls=[1, 5, 10, 20, 30, 60, 75, 80])
smoothed['cumpnl'].plot(figsize=(8, 6))
#print('smoothed PNL\n', smoothed['sharpe'])
plt.grid()
plt.title('smoothed pnl')
plt.show()

#tilt_timing = bt_metrics.tilt_timing(tilt_rolling_window=1*244) # default 3 years  tilt_rolling_window = 3 * 244 

seasonal_pnl = bt_metrics.seasonal_pnl()
cumpnl = seasonal_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('seasonal sharpe stats\n', seasonal_pnl['sharpe_stats'])
plt.grid()
plt.title('monthly pnl')
plt.show()


monthday_pnl = bt_metrics.monthday_pnl()
cumpnl = monthday_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('monthday sharpe stats\n', monthday_pnl['sharpe_stats'])
plt.grid()
plt.title('monthday pnl')
plt.show()


week_pnl = bt_metrics.week_pnl()
cumpnl = week_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('week sharpe stats\n', week_pnl['sharpe_stats'])
plt.grid()
plt.title('weekday pnl')
plt.show()


annual_pnl = bt_metrics.annual_pnl()
cumpnl = annual_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('annual sharpe stats\n', annual_pnl['sharpe_stats'])
plt.grid()
plt.title('annual pnl')
plt.show()

annual_pnl['cumlog_pnl'].mean(axis=1).plot()
plt.grid()
plt.title('annual averaged profile')
plt.show()

# turnover = bt_metrics.turnover()
# print(turnover)

# batch feature exploration

In [ ]:
feature_list = [
#     'margin_hrc_sh', 
    'strip_hsec',
    'strip_3.0x685',
    'pipe_1.5x3.25',    
    'hrc_sh',
    'crc_sh',
    'billet_ts',
    'macf_cfd',
#     'gi_0.5_sh',
#     'hsec_400x200',
#     'highwire_6.5',
#     'angle_50x5',
#     'ibeam_25',
#     'channel_16',

#     'import_arb', 'pbf_prem', 'plt65_62',
#     'io_laytime_45ports', 'io_inv_imp_31ports',
#     'io_invdays_imp_mill(64)', 'io_inv_mill(64)', 'io_inv_imp_mill',
#     'io_removal_port_41',
#     'io_loading_14ports_ausbzl',
]

udf = spot_df[feature_list].dropna(how='all')
# lunar_seasonal = True

# if lunar_seasonal:
#     seasonal_signal = tstool.lunar_label(udf)
#     seasonal_signal = tstool.seasonal_group_score(
#         seasonal_signal, score_cols=feature_list, yr_col='lunar_cny',
#         group_col='lunar_wks', min_obs=3, backward=2, forward=2, rolling_years=3)
#     seasonal_signal = seasonal_signal.reindex(index=df.index).ffill()

for feature in udf.columns:
    dataseries.plot_seasonal_df(udf[feature].dropna(), cutoff='2018-01-01', title=feature)
    
signal_raw = udf[feature_list].reindex(index=df.index).ffill()


In [ ]:
cutoff = '2012-01-01'
signal_func = 'qtl'
param_rng = [20, 42, 2]
signal_cap = None # [-2, 2]
product_list = ['rb', 'hc', 'i', 'j', 'jm', 'SF', 'FG', ] # 'v', 'cu', 'al', 'ss', 'UR', 'SA', 'ru'

for asset in product_list:
    if '_' in asset:
        price_ts = (1 + beta_ret_dict[asset]).cumprod().to_frame('price')[cutoff:]
    else:
        price_ts = df[(asset, 'c1', 'close')].dropna().to_frame('price')[cutoff:]
    pnl_list = [price_ts]
    for feature in feature_list:
        feature_ts = udf[feature].reindex(index=price_ts.index).ffill()
        #feature_ts = feature_ts.pct_change(5)
        #feature_ts = tstool.lunar_yoy(feature_ts, group_col='lunar_days', func='pct_change')
        #feature_ts = tstool.seasonal_score(feature_ts.to_frame())
        signal_ts = calc_conv_signal(feature_ts, signal_func=signal_func, param_rng=param_rng, signal_cap=signal_cap)
        asset_df = pd.concat([price_ts, signal_ts], axis=1)
        asset_df.columns = ['price', 'signal']
        asset_df['signal'] = asset_df['signal'].apply(lambda x: x).ffill()
        asset_df = asset_df.dropna(subset=['price'])
        asset_df['position'] = (asset_df['signal']/asset_df['price'].pct_change().rolling(20).std()).shift(1).fillna(0)
        asset_df['pnl'] = (asset_df['position'].shift(1) * asset_df['price'].pct_change()).fillna(0)
        
        sr = np.sqrt(244) * asset_df['pnl'].mean()/asset_df['pnl'].std()
        pnl_per_trade = 100 * 100 * asset_df['pnl'].mean()/asset_df['position'].diff().abs().mean()
        turnover = 100 * asset_df['position'].diff().abs().mean()/asset_df['position'].abs().mean()
        print(f'{asset}:{feature} -> SR: {sr:.2f} -- PNL per trade: {pnl_per_trade:.2f} -- Turnover: {turnover:.2f}')
        pnl_list.append(asset_df['pnl'].cumsum().to_frame(feature))
    pnl_df = pd.concat(pnl_list, axis=1)
    dataseries.plot_df_on_2ax(pnl_df, left_on=feature_list, right_on=['price'])
    

# signal grid search run

In [ ]:
signal_list = []

for feature in feature_list:
    for win in [20, 40, 60, 80, 120, 240]:
        signal_name = f"{feature}:ma:{win}"
        signal_raw[signal_name] = signal_raw[feature] - signal_raw[feature].rolling(win).mean()
        signal_raw[signal_name] = dh.risk_normalized(signal_raw[signal_name], 60)
        signal_list.append(signal_name)
        
        signal_name = f"{feature}:ewmac:{win}"
        signal_raw[signal_name] = dh.ewmac(signal_raw[feature], win_s=win/10, ls_ratio=2)
        signal_raw[signal_name] = dh.risk_normalized(signal_raw[signal_name], 60)
        signal_list.append(signal_name)

#         signal_name = f"{feature}:convewm:{win}"
#         signal_raw[signal_name] = dh.conv_ewm(signal_raw[feature], h1s=[win//10, win//10*2], h2s=[win//10*3, win//10*6])
#         signal_raw[signal_name] = dh.risk_normalized(signal_raw[signal_name], 60)
#         signal_list.append(signal_name)
        
        signal_name = f"{feature}:zscore:{win}"
        signal_raw[signal_name] = dh.zscore_roll(signal_raw[feature], win=win)
        signal_list.append(signal_name) 
        
        signal_name = f"{feature}:zscore_dff20:{win}"
        signal_raw[signal_name] = dh.zscore_roll(signal_raw[feature].diff(20), win=win)
        signal_list.append(signal_name) 
        
        signal_name = f"{feature}:qtl:{win}"
        signal_raw[signal_name] = dh.pct_score(signal_raw[feature], win=win)*2
        signal_list.append(signal_name) 
        
        signal_name = f"{feature}:qtl_dff20:{win}"
        signal_raw[signal_name] = dh.pct_score(signal_raw[feature].diff(20), win=win)*2
        signal_list.append(signal_name)
        
        signal_name = f"{feature}:lunar_wks_score:{win}"
        signal_raw[signal_name] = seasonal_signal[feature]
        signal_list.append(signal_name)
        
        signal_prefix = f"{feature}:seasonal_score"
        signal_raw[signal_prefix] = tstool.seasonal_score(signal_raw[feature].to_frame())
        signal_name = f"{signal_prefix}_pct:{win}"
        signal_raw[signal_name] = dh.pct_score(signal_raw[feature], win=win)*2
        signal_list.append(signal_name) 
        signal_name = f"{signal_prefix}_zscore:{win}"
        signal_raw[signal_name] = dh.zscore_roll(signal_raw[feature], win=win)
        signal_list.append(signal_name)
        
        signal_prefix = f"{feature}:yoy"
        signal_raw[signal_prefix] = signal_raw[feature]/signal_raw[feature].shift(244)-1
        signal_name = f"{signal_prefix}_pct:{win}"
        signal_raw[signal_name] = dh.pct_score(signal_raw[feature], win=win)*2
        signal_list.append(signal_name) 
        signal_name = f"{signal_prefix}_zscore:{win}"
        signal_raw[signal_name] = dh.zscore_roll(signal_raw[feature], win=win)
        signal_list.append(signal_name)

        signal_prefix = f"{feature}:lunar_yoy"
        signal_raw[signal_prefix] = tstool.lunar_yoy(signal_raw[feature], group_col='lunar_days', func='pct_change')
        signal_name = f"{signal_prefix}_pct:{win}"
        signal_raw[signal_name] = dh.pct_score(signal_raw[feature], win=win)*2
        signal_list.append(signal_name) 
        signal_name = f"{signal_prefix}_zscore:{win}"
        signal_raw[signal_name] = dh.zscore_roll(signal_raw[feature], win=win)
        signal_list.append(signal_name)
        
signal_raw = signal_raw.reindex(index=df.index).ffill()

In [ ]:
product_list = ['rb', 'hc', 'i', 'j', 'jm', 'v', 'FG', 'SM', 'SF']
cutoff = pd.Timestamp('2012-07-01')

for sig in signal_list:
    print(sig)
    pnl_by_asset = {}
    pnl_df = pd.DataFrame()
    pos_df = pd.DataFrame()
    for asset in product_list:
        signal = signal_raw[sig]
        asset_df = pd.concat([df[(asset, 'c1', 'close')], signal], axis=1)
        asset_df.columns = ['price', 'signal']
        asset_df['signal'] = asset_df['signal'].apply(lambda x: x).ffill()
        asset_df = asset_df.dropna(subset=['price']).ffill()
        asset_df['position'] = (asset_df['signal']/asset_df['price'].pct_change().rolling(20).std()).shift(1).fillna(0)
        asset_df['pnl'] = (asset_df['position'].shift(1) * asset_df['price'].pct_change()).fillna(0)
        
        sr = np.sqrt(244) * asset_df['pnl'].mean()/asset_df['pnl'].std()
        pnl_per_trade = 100 * 100 * asset_df['pnl'].mean()/asset_df['position'].diff().abs().mean()
        turnover = 100 * asset_df['position'].diff().abs().mean()/asset_df['position'].abs().mean()
        print(f'{asset} -> SR: {sr:.2f} -- PNL per trade: {pnl_per_trade:.2f} -- Turnover: {turnover:.2f}')
        
        pnl_by_asset[asset] = asset_df
        pnl_df[asset] = asset_df['pnl']
        pos_df[asset] = asset_df['position']
    pnl_df = pnl_df.fillna(0)
    pos_df = pos_df.ffill()
    total_sr = = np.sqrt(244) * pnl_df.sum(axis=1).mean()/pnl_df.sum(axis=1).std()
    print(f'Total SR: {total_sr:.2f}')
    
    cumpn; = pnl_df.cumsum()
    cumpnl.plot()
    plt.title(sig)
    plt.show()
    cum_pnl.sum(axis=1).plot()
    plt.title(sig)
    plt.show()
    

# Signal portfolio

In [ ]:
signal_dict_full = {
    'i': [
        ('io_removal_lvl_fast', 1.0), 
        #('io_removal_lyoy_mom', 1.0),
        
        ('io_inv_mill(64)_lvl_fast', 0.5),
        #('io_inv_mill(64)_lyoy_mom', 0.5),
        
        ('io_invdays_imp_mill(64)_lvl_fast', 0.5),
        #('io_invdays_imp_mill(64)_lyoy_mom', 0.5),
        
#         ('steel_social_inv_lvl_fast', 1.0/1.0),
#         ('rebar_inv_social_lyoy_fast', 0.25/1.0),
#         ('wirerod_inv_social_lyoy_fast', 0.25/1.0),
#         ('hrc_inv_social_lyoy_fast', 0.25/1.0),
#         ('crc_inv_social_lyoy_fast', 0.25/1.0),
        
        ('margin_lvl_fast', 1.0),
        ('strip_hsec_lvl_mid', 1.0),
        ('macf_cfd_lvl_mid', 1.0),
#         ('pbf_prem_yoy', 0.5/15),
#         ('cons_steel_lyoy_slow', 1.0/1.5),
#         ('sea_export_arb_lvl_mid', 1.0/1.4),
    ],
    'rb': [
        ('io_removal_lvl_fast', 1.0), 
        #('io_removal_lyoy_mom', 1.0),
        
        ('io_inv_mill(64)_lvl_fast', 1.0),
        #('io_inv_mill(64)_lyoy_mom', 1.0),
        
#         ('rebar_inv_social_lyoy_fast', 1.0),
#         ('wirerod_inv_social_lyoy_fast', 1.0),
        
        ('margin_lvl_fast', 1.0),
        ('strip_hsec_lvl_mid', 1.0),    
    ],
    'hc': [
        ('io_removal_lvl_fast', 1.0), 
        #('io_removal_lyoy_mom', 1.0),
        
        ('io_inv_mill(64)_lvl_fast', 1.0),
        #('io_inv_mill(64)_lyoy_mom', 1.0),
        
#         ('hrc_inv_social_lyoy_fast', 1.0),
#         ('crc_inv_social_lyoy_fast', 1.0),        
        ('margin_lvl_fast', 1.0),
        ('strip_hsec_lvl_mid', 1.0),  
    ],
    'j': [
        #('io_removal_lvl_fast', 1.0), 
        #('io_removal_lyoy_mom', 1.0),
        
        #('io_inv_mill(64)_lvl_fast', 1.0),
        #('io_inv_mill(64)_lyoy_mom', 1.0),
        
#         ('steel_social_inv_lvl_fast', 1.0),
        
        ('margin_lvl_fast', 1.0),
        ('strip_hsec_lvl_mid', 1.0),    
    ],
    'jm': [
        #('io_inv_mill(64)_lvl_fast', 1.0),
        #('io_inv_mill(64)_lyoy_mom', 1.0),
        
#         ('steel_social_inv_lvl_fast', 1.0),
        
        ('margin_lvl_fast', 1.0),
        ('strip_hsec_lvl_mid', 1.0),     
    ],
    'FG': [
        #('io_inv_mill(64)_lvl_fast', 1.0),
        #('io_inv_mill(64)_lyoy_mom', 1.0),
        
        ('margin_lvl_fast', 1.0),
        ('strip_hsec_lvl_mid', 1.0),    
    ],
}

In [ ]:
signal_dict = signal_dict_full

signal_diagnosis = False

pnl_dict = {}
pos_dict = {}

for asset in ['i', 'rb', 'hc', 'j', 'jm']:
    if '_' in asset:
        price_ts = (1 + beta_ret_dict[asset]).cumprod().to_frame('price')
    else:
        price_ts = df[(asset, 'c1', 'close')].dropna().to_frame('price')
    pnl_list = []
    pos_list = []
    for idx, (feature_name, weight) in enumerate(signal_dict[asset]):
        feature, signal_func, param_rng, proc_func, chg_func, bullish, freq = signal_repo[feature_name]
        if freq == 'price':
            feature_ts = spot_df[feature].ffill().reindex(index=price_ts.index).ffill()
        elif len(freq) > 0:
            feature_ts = spot_df[feature].ffill().reindex(index=pd.date_range(start=df.index[0], end=df.index[-1], freq=freq)).ffill()
        else:
            feature_ts = spot_df[feature].dropna()
        
        if 'yoy' in proc_func:
            if 'lunar' in proc_func:
                label_func = lunar_label
                label_args = {}
            else:
                label_func = calendar_label
                label_args = {'anchor_date': {'month': 1, 'day': 1}}
            if '_wk' in proc_func:
                group_col = 'label_wk'
            else:
                group_col = 'label_day'
            feature_ts = yoy_generic(feature_ts, label_func=label_func, group_col='label_day', func=chg_func, label_args=label_args)
        elif 'df' in proc_func:
            n_diff = int(proc_func[2:])
            feature_ts = getattr(feature_ts, chg_func)(n_diff)
    
        if signal_func == 'seasonal_score_w':
            signal_ts = seasonal_score(feature_ts.to_frame(), backward=10, forward=10, rolling_years=3, min_obs=10).reindex(index=df.index).ffill()
        elif signal_func == 'seasonal_score_d':
            signal_ts = seasonal_score(feature_ts.to_frame(), backward=15, forward=15, rolling_years=3, min_obs=30)
        elif len(signal_func)>0:
            feature_ts = feature_ts.reindex(index=df.index).ffill()
            signal_ts = calc_conv_signal(feature_ts, signal_func=signal_func, param_rng=param_rng, signal_cap=signal_cap)
        else:
            signal_ts = feature_ts.reindex(index=df.index).ffill()
            
        if not bullish:
            signal_ts = -signal_ts
            
        asset_df = pd.concat([price_ts, signal_ts], axis=1)
        asset_df.columns = ['price', 'signal']
        asset_df['signal'] = asset_df['signal'].apply(lambda x: x).ffill()
        asset_df = asset_df.dropna(subset=['price'])
        asset_df['position'] = (weight*asset_df['signal']/asset_df['price'].pct_change().rolling(20).std()).shift(1).fillna(0)
        asset_df['pnl'] = (asset_df['position'].shift(1) * asset_df['price'].pct_change()).fillna(0)
        
        std = asset_df['pnl'].std()
        sr = np.sqrt(244) * asset_df['pnl'].mean()/asset_df['pnl'].std()
        pnl_per_trade = 100 * 100 * asset_df['pnl'].mean()/asset_df['position'].diff().abs().mean()
        turnover = 100 * asset_df['position'].diff().abs().mean()/asset_df['position'].abs().mean()
        print(f'{asset}:{feature_name} -> SR: {sr:.2f} -- PNL per trade: {pnl_per_trade:.2f} -- Turnover: {turnover:.2f}')
        pnl_list.append(asset_df['pnl'].to_frame(feature_name))
        pos_list.append(asset_df['position'].to_frame(feature_name))
        
    pnl_df = pd.concat(pnl_list, axis=1)
    pos_df = pd.concat(pos_list, axis=1)
    sum_pnl = pnl_df.sum(axis=1)
    sum_pos = pos_df.sum(axis=1)
    sr = np.sqrt(244) * sum_pnl.mean()/sum_pnl.std()
    pnl_per_trade = 100 * 100 * sum_pnl.mean()/sum_pos.diff().abs().mean()
    turnover = 100 * sum_pos.diff().abs().mean()/sum_pos.abs().mean()
    print(f'{asset}:total -> SR: {sr:.2f} -- PNL per trade: {pnl_per_trade:.2f} -- Turnover: {turnover:.2f}')
    
    print(pnl_df.std())
    pnl_dict[asset] = pnl_df
    pos_dict[asset] = pos_df
    pnl_df.cumsum().plot()
    plt.show()
    sum_pnl.cumsum().plot()
    plt.show()


In [ ]:
fill_backward = False
smooth_win = 1
sig_smooth = tstool.exp_smooth(df_in, hl = smooth_win, fill_backward=fill_backward)

demean = False
mean_win = 244
vol_win = 244
if demean:
    sig_scored = tstool.ts_score(sig_smooth, hl_mean=mean_win, min_obs_mean=mean_win, fill_backward_mean=fill_backward, 
                          hl_vol=vol_win, min_obs_vol=vol_win, fill_backward_vol=fill_backward)
else:
    sig_scored = tstool.ts_scale(sig_smooth, hl = vol_win, min_obs=vol_win, fill_backward=fill_backward)

#sig_scored = tstool.xs_score(sig_smooth, demean=demean, hl=vol_win)

signal_cap = 2.0

score_capped = tstool.cap(sig_scored, -signal_cap, signal_cap)
score_filled = tstool.filldown(score_capped, 2)
score = tstool.lag(score_filled, 1)


In [ ]:
vol_scale = 20
asset_vol = tstool.exp_smooth(df_pxchg**2, hl=vol_scale, fill_backward=fill_backward)**0.5
holding = score/asset_vol

commod_list = holding.columns #['hc']
btmetrics = MetricsBase(holdings = holding[commod_list], returns = df_pxchg[commod_list])